# Strategy Optimization Notebook

This notebook focuses on optimizing trading strategies and analyzing their performance under different parameters.

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import yaml
from datetime import datetime
from itertools import product

# Add src directory to path
import sys
sys.path.append('../')

# Import our modules
from src.data_loader import fetch_data
from src.strategy import MomentumStrategy, MeanReversionStrategy, MovingAverageCrossoverStrategy, MACDStrategy, EnsembleStrategy
from src.backtest import BacktestEngine
from src.visualization import plot_portfolio_performance, plot_performance_table

# Set up plotting style
plt.style.use('ggplot')
sns.set_style('whitegrid')
%matplotlib inline

## Load Configuration & Data

In [ ]:
# Load configuration
with open('../config/config.yaml', 'r') as f:
    config = yaml.safe_load(f)

# Fetch data
ticker = config['data']['tickers'][1]  # S&P500
start_date = config['data']['start_date']
end_date = config['data']['end_date']

print(f"Fetching data for {ticker} from {start_date} to {end_date}")
data = fetch_data(ticker, start_date, end_date)

# Add High, Low, Close columns if needed
if 'High' not in data.columns or 'Low' not in data.columns or 'Close' not in data.columns:
    data['High'] = data['Price']
    data['Low'] = data['Price']
    data['Close'] = data['Price']

# Split data for training and testing
split_index = int(0.8 * len(data))
train_data = data.iloc[:split_index]
test_data = data.iloc[split_index:]

print(f"Training data: {train_data.shape} rows")
print(f"Testing data: {test_data.shape} rows")

## Optimize Momentum Strategy

In [ ]:
# Define parameter grid for momentum strategy
momentum_windows = [5, 10, 12, 15, 20, 30, 60]
momentum_thresholds = [0.0, 0.005, 0.01, 0.02, 0.03]

# Initialize results storage
momentum_results = []

# Grid search
for window, threshold in product(momentum_windows, momentum_thresholds):
    # Create strategy with these parameters
    strategy_config = config.copy()
    strategy_config['momentum_window'] = window
    strategy_config['momentum_threshold'] = threshold
    
    strategy = MomentumStrategy(strategy_config)
    
    # Run backtest
    backtest = BacktestEngine(test_data, strategy, strategy_config)
    backtest.run()
    
    # Extract key metrics
    metrics = backtest.performance_metrics
    
    # Store results
    result = {
        'window': window,
        'threshold': threshold,
        'sharpe_ratio': metrics['sharpe_ratio_strategy'],
        'annual_return': metrics['annual_return_strategy'],
        'max_drawdown': metrics['max_drawdown_strategy'],
        'win_rate': metrics.get('win_rate', 0),
        'total_trades': metrics.get('total_trades', 0)
    }
    
    momentum_results.append(result)
    
    # Print progress
    print(f"Window: {window}, Threshold: {threshold}, Sharpe: {result['sharpe_ratio']:.2f}, Return: {result['annual_return']:.2%}")

# Convert to DataFrame
momentum_df = pd.DataFrame(momentum_results)

# Sort by Sharpe ratio (descending)
momentum_df = momentum_df.sort_values('sharpe_ratio', ascending=False)

# Display top 5 results
print("\nTop 5 Momentum Strategy Parameters:")
momentum_df.head(5)

## Optimize Mean Reversion Strategy

In [ ]:
# Define parameter grid for mean reversion strategy
mean_rev_windows = [10, 15, 20, 30, 40, 50]
mean_rev_thresholds = [0.5, 1.0, 1.5, 2.0, 2.5, 3.0]

# Initialize results storage
mean_rev_results = []

# Grid search
for window, threshold in product(mean_rev_windows, mean_rev_thresholds):
    # Create strategy with these parameters
    strategy_config = config.copy()
    strategy_config['mean_reversion_window'] = window
    strategy_config['mean_reversion_threshold'] = threshold
    
    strategy = MeanReversionStrategy(strategy_config)
    
    # Run backtest
    backtest = BacktestEngine(test_data, strategy, strategy_config)
    backtest.run()
    
    # Extract key metrics
    metrics = backtest.performance_metrics
    
    # Store results
    result = {
        'window': window,
        'threshold': threshold,
        'sharpe_ratio': metrics['sharpe_ratio_strategy'],
        'annual_return': metrics['annual_return_strategy'],
        'max_drawdown': metrics['max_drawdown_strategy'],
        'win_rate': metrics.get('win_rate', 0),
        'total_trades': metrics.get('total_trades', 0)
    }
    
    mean_rev_results.append(result)
    
    # Print progress
    print(f"Window: {window}, Threshold: {threshold}, Sharpe: {result['sharpe_ratio']:.2f}, Return: {result['annual_return']:.2%}")

# Convert to DataFrame
mean_rev_df = pd.DataFrame(mean_rev_results)

# Sort by Sharpe ratio (descending)
mean_rev_df = mean_rev_df.sort_values('sharpe_ratio', ascending=False)

# Display top 5 results
print("\nTop 5 Mean Reversion Strategy Parameters:")
mean_rev_df.head(5)

## Visualize Parameter Impact on Performance

In [ ]:
# Create heatmaps to visualize parameter impact
plt.figure(figsize=(20, 10))

# Momentum Strategy - Sharpe Ratio
plt.subplot(2, 2, 1)
pivot_momentum = momentum_df.pivot_table(index='window', columns='threshold', values='sharpe_ratio')
sns.heatmap(pivot_momentum, annot=True, cmap='viridis', fmt='.2f')
plt.title('Momentum Strategy: Sharpe Ratio')
plt.xlabel('Momentum Threshold')
plt.ylabel('Window Size')

# Momentum Strategy - Annual Return
plt.subplot(2, 2, 2)
pivot_momentum = momentum_df.pivot_table(index='window', columns='threshold', values='annual_return')
sns.heatmap(pivot_momentum, annot=True, cmap='viridis', fmt='.2%')
plt.title('Momentum Strategy: Annual Return')
plt.xlabel('Momentum Threshold')
plt.ylabel('Window Size')

# Mean Reversion Strategy - Sharpe Ratio
plt.subplot(2, 2, 3)
pivot_mean_rev = mean_rev_df.pivot_table(index='window', columns='threshold', values='sharpe_ratio')
sns.heatmap(pivot_mean_rev, annot=True, cmap='viridis', fmt='.2f')
plt.title('Mean Reversion Strategy: Sharpe Ratio')
plt.xlabel('Z-score Threshold')
plt.ylabel('Window Size')

# Mean Reversion Strategy - Annual Return
plt.subplot(2, 2, 4)
pivot_mean_rev = mean_rev_df.pivot_table(index='window', columns='threshold', values='annual_return')
sns.heatmap(pivot_mean_rev, annot=True, cmap='viridis', fmt='.2%')
plt.title('Mean Reversion Strategy: Annual Return')
plt.xlabel('Z-score Threshold')
plt.ylabel('Window Size')

plt.tight_layout()
plt.show()

# Scatter plot of returns vs drawdowns
plt.figure(figsize=(15, 6))

# Momentum Strategy
plt.subplot(1, 2, 1)
plt.scatter(momentum_df['max_drawdown'], momentum_df['annual_return'], 
            c=momentum_df['sharpe_ratio'], cmap='viridis', 
            s=100, alpha=0.7)
plt.colorbar(label='Sharpe Ratio')
plt.title('Momentum Strategy: Return vs Drawdown')
plt.xlabel('Maximum Drawdown')
plt.ylabel('Annual Return')
plt.grid(True)

# Mean Reversion Strategy
plt.subplot(1, 2, 2)
plt.scatter(mean_rev_df['max_drawdown'], mean_rev_df['annual_return'], 
            c=mean_rev_df['sharpe_ratio'], cmap='viridis', 
            s=100, alpha=0.7)
plt.colorbar(label='Sharpe Ratio')
plt.title('Mean Reversion Strategy: Return vs Drawdown')
plt.xlabel('Maximum Drawdown')
plt.ylabel('Annual Return')
plt.grid(True)

plt.tight_layout()
plt.show()

# Save the best parameters for later use
best_momentum_params = momentum_df.iloc[0].to_dict()
best_mean_rev_params = mean_rev_df.iloc[0].to_dict()

print(f"Best Momentum Parameters: Window={best_momentum_params['window']}, Threshold={best_momentum_params['threshold']}")
print(f"Best Mean Reversion Parameters: Window={best_mean_rev_params['window']}, Threshold={best_mean_rev_params['threshold']}")

In [ ]:
def optimize_ma_strategy(test_data, config):
    """Optimize Moving Average Crossover Strategy"""
    # Define parameter grid
    fast_windows = [10, 20, 50, 100]
    slow_windows = [50, 100, 200, 300]
    
    # Only test valid combinations (fast < slow)
    ma_results = []
    
    for fast, slow in [(f, s) for f in fast_windows for s in slow_windows if f < s]:
        # Create strategy with these parameters
        strategy_config = config.copy()
        strategy_config['fast_ma_window'] = fast
        strategy_config['slow_ma_window'] = slow
        
        strategy = MovingAverageCrossoverStrategy(strategy_config)
        
        # Run backtest
        backtest = BacktestEngine(test_data, strategy, strategy_config)
        backtest.run()
        
        # Extract key metrics
        metrics = backtest.performance_metrics
        
        # Store results
        result = {
            'fast_window': fast,
            'slow_window': slow,
            'sharpe_ratio': metrics['sharpe_ratio_strategy'],
            'annual_return': metrics['annual_return_strategy'],
            'max_drawdown': metrics['max_drawdown_strategy'],
            'win_rate': metrics.get('win_rate', 0),
            'total_trades': metrics.get('total_trades', 0)
        }
        
        ma_results.append(result)
        
        # Print progress
        print(f"Fast MA: {fast}, Slow MA: {slow}, Sharpe: {result['sharpe_ratio']:.2f}, Return: {result['annual_return']:.2%}")
    
    # Convert to DataFrame
    ma_df = pd.DataFrame(ma_results)
    
    # Sort by Sharpe ratio (descending)
    ma_df = ma_df.sort_values('sharpe_ratio', ascending=False)
    
    return ma_df

def optimize_macd_strategy(test_data, config):
    """Optimize MACD Strategy"""
    # Define parameter grid
    fast_periods = [8, 10, 12, 15]
    slow_periods = [20, 26, 30]
    signal_periods = [7, 9, 12]
    
    # Initialize results storage
    macd_results = []
    
    for fast, slow, signal in product(fast_periods, slow_periods, signal_periods):
        if fast >= slow:  # Skip invalid combinations
            continue
            
        # Create strategy with these parameters
        strategy_config = config.copy()
        strategy_config['macd_fast_period'] = fast
        strategy_config['macd_slow_period'] = slow
        strategy_config['macd_signal_period'] = signal
        
        strategy = MACDStrategy(strategy_config)
        
        # Run backtest
        backtest = BacktestEngine(test_data, strategy, strategy_config)
        backtest.run()
        
        # Extract key metrics
        metrics = backtest.performance_metrics
        
        # Store results
        result = {
            'fast_period': fast,
            'slow_period': slow,
            'signal_period': signal,
            'sharpe_ratio': metrics['sharpe_ratio_strategy'],
            'annual_return': metrics['annual_return_strategy'],
            'max_drawdown': metrics['max_drawdown_strategy'],
            'win_rate': metrics.get('win_rate', 0),
            'total_trades': metrics.get('total_trades', 0)
        }
        
        macd_results.append(result)
        
        # Print progress
        print(f"Fast: {fast}, Slow: {slow}, Signal: {signal}, Sharpe: {result['sharpe_ratio']:.2f}")
    
    # Convert to DataFrame
    macd_df = pd.DataFrame(macd_results)
    
    # Sort by Sharpe ratio (descending)
    macd_df = macd_df.sort_values('sharpe_ratio', ascending=False)
    
    return macd_df

# Run the optimizations for MA and MACD strategies
print("\nOptimizing Moving Average Crossover Strategy...")
ma_df = optimize_ma_strategy(test_data, config)
print("\nTop 5 Moving Average Strategy Parameters:")
print(ma_df.head(5))

print("\nOptimizing MACD Strategy...")
macd_df = optimize_macd_strategy(test_data, config)
print("\nTop 5 MACD Strategy Parameters:")
print(macd_df.head(5))

# Save the best parameters
best_ma_params = ma_df.iloc[0].to_dict()
best_macd_params = macd_df.iloc[0].to_dict()

print(f"\nBest MA Parameters: Fast={best_ma_params['fast_window']}, Slow={best_ma_params['slow_window']}")
print(f"Best MACD Parameters: Fast={best_macd_params['fast_period']}, Slow={best_macd_params['slow_period']}, Signal={best_macd_params['signal_period']}")

## Create and Test Ensemble Strategy

In [ ]:
# Create and Test Ensemble Strategy

# Combine the best strategies into an ensemble approach
print("\nCreating Ensemble Strategy with best parameters from each individual strategy...")

# Use the best parameters for each strategy
ensemble_config = config.copy()
ensemble_config.update({
    # Momentum strategy parameters
    'momentum_window': best_momentum_params['window'],
    'momentum_threshold': best_momentum_params['threshold'],
    
    # Mean Reversion strategy parameters
    'mean_reversion_window': best_mean_rev_params['window'],
    'mean_reversion_threshold': best_mean_rev_params['threshold'],
    
    # Moving Average strategy parameters
    'fast_ma_window': best_ma_params['fast_window'],
    'slow_ma_window': best_ma_params['slow_window'],
    
    # MACD strategy parameters
    'macd_fast_period': best_macd_params['fast_period'],
    'macd_slow_period': best_macd_params['slow_period'],
    'macd_signal_period': best_macd_params['signal_period'],
})

# Create individual strategies with optimized parameters
momentum_strategy = MomentumStrategy(ensemble_config)
mean_reversion_strategy = MeanReversionStrategy(ensemble_config)
ma_strategy = MovingAverageCrossoverStrategy(ensemble_config)
macd_strategy = MACDStrategy(ensemble_config)

# List of strategies for ensemble
strategies = [
    momentum_strategy,
    mean_reversion_strategy,
    ma_strategy,
    macd_strategy
]

# Try different weighting schemes for the ensemble
print("Testing different weighting schemes for the ensemble strategy...")

# 1. Equal weighting
equal_weights = {strategy.name: 1/len(strategies) for strategy in strategies}
print(f"Equal weights: {equal_weights}")

# 2. Performance-based weighting (based on Sharpe ratio)
performance_metrics = {
    momentum_strategy.name: best_momentum_params['sharpe_ratio'],
    mean_reversion_strategy.name: best_mean_rev_params['sharpe_ratio'],
    ma_strategy.name: best_ma_params['sharpe_ratio'],
    macd_strategy.name: best_macd_params['sharpe_ratio']
}

# Normalize weights to sum to 1
total_performance = sum(performance_metrics.values())
performance_weights = {
    name: value / total_performance for name, value in performance_metrics.items()
}
print(f"Performance-based weights: {performance_weights}")

# 3. Risk-adjusted weighting (inversely proportional to volatility)
risk_metrics = {
    momentum_strategy.name: best_momentum_params['max_drawdown'],
    mean_reversion_strategy.name: best_mean_rev_params['max_drawdown'],
    ma_strategy.name: best_ma_params['max_drawdown'],
    macd_strategy.name: best_macd_params['max_drawdown']
}

# Higher drawdown means higher risk, so we use inverse
risk_weights = {
    name: 1 / (abs(value) + 0.001) for name, value in risk_metrics.items()
}

# Normalize weights to sum to 1
total_risk_weight = sum(risk_weights.values())
risk_weights = {
    name: value / total_risk_weight for name, value in risk_weights.items()
}
print(f"Risk-adjusted weights: {risk_weights}")

# Create ensemble strategies with different weighting schemes
equal_ensemble = EnsembleStrategy(strategies, equal_weights, ensemble_config)
performance_ensemble = EnsembleStrategy(strategies, performance_weights, ensemble_config)
risk_ensemble = EnsembleStrategy(strategies, risk_weights, ensemble_config)

# Test each ensemble strategy
print("\nTesting Equal-Weighted Ensemble...")
equal_backtest = BacktestEngine(test_data, equal_ensemble, ensemble_config)
equal_backtest.run()
equal_metrics = equal_backtest.performance_metrics

print("\nTesting Performance-Weighted Ensemble...")
performance_backtest = BacktestEngine(test_data, performance_ensemble, ensemble_config)
performance_backtest.run()
performance_metrics = performance_backtest.performance_metrics

print("\nTesting Risk-Adjusted Ensemble...")
risk_backtest = BacktestEngine(test_data, risk_ensemble, ensemble_config)
risk_backtest.run()
risk_metrics = risk_backtest.performance_metrics

# Create comparison table
ensemble_results = pd.DataFrame({
    'Equal-Weighted': {
        'Sharpe Ratio': equal_metrics['sharpe_ratio_strategy'],
        'Annual Return': equal_metrics['annual_return_strategy'],
        'Max Drawdown': equal_metrics['max_drawdown_strategy'],
        'Win Rate': equal_metrics['win_rate'],
        'Total Trades': equal_metrics['total_trades']
    },
    'Performance-Weighted': {
        'Sharpe Ratio': performance_metrics['sharpe_ratio_strategy'],
        'Annual Return': performance_metrics['annual_return_strategy'],
        'Max Drawdown': performance_metrics['max_drawdown_strategy'],
        'Win Rate': performance_metrics['win_rate'],
        'Total Trades': performance_metrics['total_trades']
    },
    'Risk-Adjusted': {
        'Sharpe Ratio': risk_metrics['sharpe_ratio_strategy'],
        'Annual Return': risk_metrics['annual_return_strategy'],
        'Max Drawdown': risk_metrics['max_drawdown_strategy'],
        'Win Rate': risk_metrics['win_rate'],
        'Total Trades': risk_metrics['total_trades']
    }
})

# Display ensemble comparison
print("\nEnsemble Strategy Comparison:")
ensemble_results_display = ensemble_results.copy()
ensemble_results_display.loc['Annual Return'] = ensemble_results_display.loc['Annual Return'].map('{:.2%}'.format)
ensemble_results_display.loc['Max Drawdown'] = ensemble_results_display.loc['Max Drawdown'].map('{:.2%}'.format)
ensemble_results_display.loc['Win Rate'] = ensemble_results_display.loc['Win Rate'].map('{:.2%}'.format)
print(ensemble_results_display)

# Find the best ensemble strategy
best_ensemble_sharpe = ensemble_results.loc['Sharpe Ratio'].max()
best_ensemble_type = ensemble_results.loc['Sharpe Ratio'].idxmax()

print(f"\nThe best ensemble strategy is {best_ensemble_type} with a Sharpe ratio of {best_ensemble_sharpe:.2f}")

# Plot comparison of individual strategies vs. best ensemble
plt.figure(figsize=(14, 7))

# Get the right backtest object based on best ensemble type
if best_ensemble_type == 'Equal-Weighted':
    best_ensemble_backtest = equal_backtest
elif best_ensemble_type == 'Performance-Weighted':
    best_ensemble_backtest = performance_backtest
else:
    best_ensemble_backtest = risk_backtest

# Plot cumulative returns
plt.subplot(1, 2, 1)
cumulative_returns = pd.DataFrame({
    'Momentum': momentum_df.iloc[0]['annual_return'],
    'Mean Reversion': mean_rev_df.iloc[0]['annual_return'],
    'Moving Average': ma_df.iloc[0]['annual_return'],
    'MACD': macd_df.iloc[0]['annual_return'],
    best_ensemble_type: ensemble_results.loc['Annual Return', best_ensemble_type]
}, index=['Annual Return'])

cumulative_returns.T.plot(kind='bar', ax=plt.gca())
plt.title('Strategy Annual Returns')
plt.ylabel('Annual Return')
plt.grid(True)

# Plot Sharpe ratios
plt.subplot(1, 2, 2)
sharpe_ratios = pd.DataFrame({
    'Momentum': momentum_df.iloc[0]['sharpe_ratio'],
    'Mean Reversion': mean_rev_df.iloc[0]['sharpe_ratio'],
    'Moving Average': ma_df.iloc[0]['sharpe_ratio'],
    'MACD': macd_df.iloc[0]['sharpe_ratio'],
    best_ensemble_type: ensemble_results.loc['Sharpe Ratio', best_ensemble_type]
}, index=['Sharpe Ratio'])

sharpe_ratios.T.plot(kind='bar', ax=plt.gca())
plt.title('Strategy Sharpe Ratios')
plt.ylabel('Sharpe Ratio')
plt.grid(True)

plt.tight_layout()
plt.show()

# Plot the equity curve of the best ensemble strategy
plt.figure(figsize=(14, 7))
best_ensemble_backtest.portfolio['Cumulative_Market'].plot(label='Market', color='blue')
best_ensemble_backtest.portfolio['Cumulative_Strategy'].plot(label=f'Ensemble ({best_ensemble_type})', color='green')
plt.title(f'Performance of {best_ensemble_type} Ensemble Strategy')
plt.xlabel('Date')
plt.ylabel('Cumulative Return')
plt.legend()
plt.grid(True)
plt.show()

# Save the best ensemble parameters
best_ensemble_weights = None
if best_ensemble_type == 'Equal-Weighted':
    best_ensemble_weights = equal_weights
elif best_ensemble_type == 'Performance-Weighted':
    best_ensemble_weights = performance_weights
else:
    best_ensemble_weights = risk_weights

print("\nBest Ensemble Configuration:")
print(f"Strategy Weights: {best_ensemble_weights}")
print(f"Component Strategy Parameters:")
print(f"  Momentum: window={best_momentum_params['window']}, threshold={best_momentum_params['threshold']}")
print(f"  Mean Reversion: window={best_mean_rev_params['window']}, threshold={best_mean_rev_params['threshold']}")
print(f"  Moving Average: fast={best_ma_params['fast_window']}, slow={best_ma_params['slow_window']}")
print(f"  MACD: fast={best_macd_params['fast_period']}, slow={best_macd_params['slow_period']}, signal={best_macd_params['signal_period']}")